### Lab 3: Variables and templates


[From Ansible doc](https://docs.ansible.com/ansible/latest/user_guide/playbooks_variables.html#define-variables-in-inventory)

Using Variables
Ansible uses variables to manage differences between systems. With Ansible, you can execute tasks and playbooks on multiple different systems with a single command. To represent the variations among those different systems, you can create variables with standard YAML syntax, including lists and dictionaries. You can define these variables in your playbooks, in your inventory, in re-usable files or roles, or at the command line. You can also create variables during a playbook run by registering the return value or values of a task as a new variable.

After you create variables, either by defining them in a file, passing them at the command line, or registering the return value or values of a task as a new variable, you can use those variables in module arguments, in conditional “when” statements, in templates, and in loops. [The ansible-examples github repository](https://github.com/ansible/ansible-examples) contains many examples of using variables in Ansible.

Once you understand the concepts and examples on this page, read about Ansible facts, which are variables you retrieve from remote systems. With Ansible you can retrieve or discover certain variables containing information about your remote systems or about Ansible itself. Variables related to remote systems are called facts. With facts, you can use the behavior or state of one system as configuration on other systems. For example, you can use the IP address of one system as a configuration value on another system. Variables related to Ansible are called magic variables.

## Creating valid variable names
Not all strings are valid Ansible variable names. A variable name can only include letters, numbers, and underscores. Python keywords or playbook keywords are not valid variable names. A variable name cannot begin with a number.

Variable names can begin with an underscore. In many programming languages, variables that begin with an underscore are private. This is not true in Ansible. Variables that begin with an underscore are treated exactly the same as any other variable. Do not rely on this convention for privacy or security.

[Defining Variables](https://docs.ansible.com/ansible/latest/user_guide/playbooks_variables.html#define-variables-in-inventory)

## Variables

In [1]:
#ansible.cfg
[defaults]
host_key_checking=False

In [2]:
#inventory
[all]
localhost ansible_connection=local

In [3]:
#vars example_vars
message: hello vars

In [4]:
#play
name: hello world
hosts: localhost
gather_facts: false
vars_files:
    - example_vars

In [5]:
#task
debug:
 msg: "{{message}}"

TASK [debug] *******************************************************************
ok: [localhost] => {
    "msg": "hello vars"
}


## Templates

In [6]:
#ansible.cfg
[defaults]
host_key_checking=False

In [7]:
#inventory
[all]
localhost ansible_connection=local

In [12]:
#vars example_vars
message: hello from template

In [13]:
#template hello.j2
{{message}} 

In [14]:
#play
name: hello world
hosts: localhost
gather_facts: false
vars_files:
    - example_vars

In [15]:
template:
    src: hello.j2
    dest: /tmp/hello

TASK [template] ****************************************************************
{'counter': 15,
 'end_line': 14,
 'event': 'verbose',
 'runner_ident': '957c3409-8c98-4579-9fc7-f377d7be878b',
 'start_line': 13,
 'stdout': '\x1b[DEPRECATION WARNING]: Distribution Ubuntu 20.04 on host '
           'localhost should use \x1b',
 'uuid': 'fb0e77ed-2a18-4ab9-89a8-9c0679b488f6'}
{'counter': 16,
 'end_line': 15,
 'event': 'verbose',
 'runner_ident': '957c3409-8c98-4579-9fc7-f377d7be878b',
 'start_line': 14,
 'stdout': '\x1b/usr/bin/python3, but is using /usr/bin/python for '
           'backward compatibility with \x1b',
 'uuid': '1be4536b-06d8-4cb7-a71b-5a3e698e5436'}
{'counter': 17,
 'end_line': 16,
 'event': 'verbose',
 'runner_ident': '957c3409-8c98-4579-9fc7-f377d7be878b',
 'start_line': 15,
 'stdout': '\x1bprior Ansible releases. A future Ansible release will '
           'default to using the \x1b',
 'uuid': '96ac48e3-a8b6-46db-b403-0653f4a5e6a7'}
{'counter': 18,
 'end_line': 17,
 'even

In [16]:
shell: cat /tmp/hello
register: output

TASK [shell] *******************************************************************
changed: [localhost] => {
    "cmd": "cat /tmp/hello",
    "delta": "0:00:00.045899",
    "end": "2022-01-17 13:33:13.300550",
    "rc": 0,
    "start": "2022-01-17 13:33:13.254651",
    "stderr": "",
    "stderr_lines": [],
    "stdout": "[see below]",
    "stdout_lines": "[removed for clarity]"
}

[localhost] stdout:
hello from template


In [17]:
debug:
    var: output.stdout

TASK [debug] *******************************************************************
ok: [localhost] => {
    "output.stdout": "hello from template"
}


## Summary:
In this lab, we discovered variabls and templates.
Let's now move on to the next topic: Roles and vaults.

* [Lab 4](4-WKSHP-Ansible101-Roles.ipynb)